In [249]:
import re
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
import pandas as pd

# FIND LINKS

In [236]:
# Configure browser options
options = webdriver.ChromeOptions()
options.add_argument('--headless')

# Set browser
driver = webdriver.Chrome(options=options)
url: str = "https://www.global-rates.com/fr/inflation/historique/"
driver.get(url)

In [237]:
# Find links to pages to scrap
url_links: list[WebElement] = driver.find_elements(By.XPATH, "//a[contains(@href, 'inflation/historique')]")
url_links: list[WebElement] = url_links[1:-1]

urls_storage: list[str] = []

for link in url_links:
    urls_storage.append(link.get_attribute("href"))

# GET INFOS FROM PAGES

In [245]:
# Set browser
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver_2 = webdriver.Chrome(options=options)

data = []

for url in urls_storage:
    driver_2.get(url)
    year = re.search(r'(\d{4})', url).group(1)
    first_table = driver_2.find_element(By.TAG_NAME, "table")
    rows = first_table.find_elements(By.TAG_NAME, "tr")

    for row in rows[1:]:
        columns = row.find_elements(By.TAG_NAME, "td")
        if len(columns) >= 4:
            country = columns[1].get_attribute("textContent").strip()
            inflation = columns[3].get_attribute("textContent").strip()
            data.append([year, country, inflation])

print(data)

driver_2.quit()


[['2025', 'Afrique du Sud', '3,16 %'], ['2025', 'Allemagne', '2,29 %'], ['2025', 'Autriche', '3,22 %'], ['2025', 'Belgique', '3,82 %'], ['2025', 'Brésil', '4,81 %'], ['2025', 'Canada', '2,27 %'], ['2025', 'Chili', '4,83 %'], ['2025', 'Chine', '-0,14 %'], ['2025', 'Corée du Sud', '2,13 %'], ['2025', 'Danemark', '1,78 %'], ['2025', 'Espagne', '2,95 %'], ['2025', 'Estonie', '4,58 %'], ['2025', 'Etats-Unis', '2,91 %'], ['2025', 'Finlande', '0,57 %'], ['2025', 'France', '1,22 %'], ['2025', 'Grèce', '2,64 %'], ['2025', 'Hongrie', '5,53 %'], ['2025', 'Inde', '3,10 %'], ['2025', 'Indonésie', '0,33 %'], ['2025', 'Irlande', '1,87 %'], ['2025', 'Islande', '4,43 %'], ['2025', 'Israël', '3,66 %'], ['2025', 'Italie', '1,54 %'], ['2025', 'Japon', '3,84 %'], ['2025', 'Luxembourg', '1,82 %'], ['2025', 'Mexique', '3,68 %'], ['2025', 'Norvège', '2,95 %'], ['2025', 'Pays-Bas', '3,56 %'], ['2025', 'Pologne', '5,34 %'], ['2025', 'Portugal', '2,46 %'], ['2025', 'Royaume-Uni', '3,80 %'], ['2025', 'Slovaquie',

In [250]:
df = pd.DataFrame(data, columns=["Année", "Pays", "Taux annuel"])
df

,Année,Pays,Taux annuel
0,2025,Afrique du Sud,"3,16 %"
1,2025,Allemagne,"2,29 %"
2,2025,Autriche,"3,22 %"
3,2025,Belgique,"3,82 %"
4,2025,Brésil,"4,81 %"
...,...,...,...
3121,1956,Portugal,"3,21 %"
3122,1956,Royaume-Uni,"5,00 %"
3123,1956,Suède,"4,97 %"
3124,1956,Suisse,"1,51 %"


In [252]:
df.to_csv("inflation_data.csv", index=False, sep=",")